# Análisis exploratorio de incidencia delictiva en México

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv('data/delitos.csv')

#### Ejercicio 1: Elige 3 estados de la república y grafica una serie de tiempo de la frecuencia abosluta de homicidios dolosos de enero 2015 a julio 2019 en estas tres entidades

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar el archivo de delitos
df = pd.read_csv('data/delitos.csv')

# Filtrar por homicidios dolosos
homi = df[df['Tipo_de_delito'] == 'Homicidio doloso']

# Filtrar por fechas
homi = homi[(homi['Año'] >= 2015) & (homi['Año'] <= 2019)]

# Elegir 3 estados
estados = ['Colima', 'Jalisco', 'Ciudad de México']
homi_estados = homi[homi['Entidad'].isin(estados)]

# Agrupar por fecha y entidad
homi_serie = homi_estados.groupby(['Año', 'Mes', 'Entidad'])['Número_de_delitos'].sum().reset_index()

# Crear columna de fecha
homi_serie['Fecha'] = pd.to_datetime(homi_serie['Año'].astype(str) + '-' + homi_serie['Mes'].astype(str))

# Graficar
plt.figure(figsize=(12,6))
for estado in estados:
    datos = homi_serie[homi_serie['Entidad'] == estado]
    plt.plot(datos['Fecha'], datos['Número_de_delitos'], label=estado)

plt.title('Homicidios dolosos (2015–2019)')
plt.xlabel('Fecha')
plt.ylabel('Número de delitos')
plt.legend()
plt.grid(True)
plt.show()


#### Ejercicio 2: Contetas las siguientes  preguntas:
1. ¿Cuántos homicidios dolosos hubo en Colima en el 2018?
2. ¿Cuantos robos de vehículo automotor ha habido en el 2019?
3. Obten la suma de homicidos dolosos y feminidios en toda la República Mexicana en cada año.
4. ¿En qué mes y en qué municipio ha ocurrido el mayor número de feminicidios?
5. ¿En qué año y en qué estado ha ocurrido el mayor número de feminicidios?

In [ ]:
colima_2018 = df[(df['Entidad'] == 'Colima') & 
                 (df['Año'] == 2018) & 
                 (df['Tipo_de_delito'] == 'Homicidio doloso')]

colima_2018['Número_de_delitos'].sum()

robo_vehiculo_2019 = df[(df['Año'] == 2019) & 
                        (df['Tipo_de_delito'] == 'Robo de vehículo automotor')]

robo_vehiculo_2019['Número_de_delitos'].sum()

delitos_sumados = df[df['Tipo_de_delito'].isin(['Homicidio doloso', 'Feminicidio'])]
suma_por_año = delitos_sumados.groupby('Año')['Número_de_delitos'].sum()
suma_por_año

fem = df[df['Tipo_de_delito'] == 'Feminicidio']
fem_max = fem.groupby(['Mes', 'Municipio'])['Número_de_delitos'].sum().reset_index()
fem_max.sort_values(by='Número_de_delitos', ascending=False).head(1)

fem_estado = fem.groupby(['Año', 'Entidad'])['Número_de_delitos'].sum().reset_index()
fem_estado.sort_values(by='Número_de_delitos', ascending=False).head(1)




#### Ejercicio 3: Haz una gráfica de pastel de tipos de delito. Deberás crear una gráfica para cada año. Utilzia la función subplots de matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Agrupar por año y tipo de delito
delitos_por_año = df.groupby(['Año', 'Tipo_de_delito'])['Número_de_delitos'].sum().reset_index()

# Obtener años únicos
años = sorted(delitos_por_año['Año'].unique())

# Crear subplots
fig, axs = plt.subplots(1, len(años), figsize=(20,6))

for i, año in enumerate(años):
    datos = delitos_por_año[delitos_por_año['Año'] == año]
    axs[i].pie(datos['Número_de_delitos'], labels=datos['Tipo_de_delito'], autopct='%1.1f%%')
    axs[i].set_title(f'Delitos en {año}')

plt.tight_layout()
plt.show()


---
#### Calcula la tasa por 100,000 habitantes
##### Tasa por 100,000 habitantes
Mostrar el total de delitos en una entidad no nos sirve de mucho. Es mucho más útil calcular la tasa de incidencia delictiva por cada 100,000 habitantes

$$
tasa = \frac{delitos\space totales}{población} \times 100,000
$$

Esta tasa la podemos anualizar multiplicándola por un factor de 12
$$
tasa\space anualizada = tasa \times 12
$$

Población por entidad federativa según [la encuesta intercensal 2015](https://www.inegi.org.mx/programas/intercensal/2015/)

No tienes que descargar nada. Ya están los datos en la carpeta data

In [ ]:
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=";")
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})
pobs.head()

In [ ]:
# Cargar población
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=';')
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})

# Agrupar delitos por entidad
delitos_totales = df.groupby('Entidad')['Número_de_delitos'].sum().reset_index()

# Unir con población
df_tasa = pd.merge(delitos_totales, pobs, left_on='Entidad', right_on='entidad')

# Calcular tasa por 100,000 habitantes
df_tasa['tasa_100k'] = (df_tasa['Número_de_delitos'] / df_tasa['poblacion']) * 100000

# Calcular tasa anualizada
df_tasa['tasa_anualizada'] = df_tasa['tasa_100k'] * 12

df_tasa[['Entidad', 'tasa_100k', 'tasa_anualizada']].head()
